In [8]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph,START,END
from langgraph.graph.message import add_messages
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate

import os
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
from agent.workflow_tools import classify,summarize,validate_requirements,get_engineer_with_lowest_load,create_ticket,route_by_classification,auto_response,send_email

In [2]:
class State(TypedDict):
    messages:Annotated[list,add_messages]

In [3]:
llm = ChatGroq(
    model_name="openai/gpt-oss-120b",   # or "llama3-70b-8192"
    groq_api_key=os.getenv("GROQ_API_KEY"),
    temperature=0.2
)

In [7]:
tools = [classify]
llm_with_tools = llm.bind_tools(tools)

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", """
        You are a Dispatcher Agent.
        Rules:
        - Create Ticket when there is an incident and the input in not a follow_up or a query.
        - Ask clarification if severity is unclear
            
        If you are unclear ask the user for clarification and end your reasoning.
        Always respond to the user either with clarification questions or the steps you took till now or both.
"""),
    ("human", "{input}")
])

chain = prompt | llm_with_tools

response = chain.invoke({
    "input": "Hello operations team , the gross sales margin in the consolidated sales report are not correct when i compare it with the SAP system, could you please check on this ASAP. This is quiet urgent since the Director needs the sales figure in this closing period. "
})

In [14]:
response

AIMessage(content='', additional_kwargs={'reasoning_content': 'We need to decide if this is an incident. It\'s about gross sales margin discrepancy, urgent. It\'s not a follow_up or query, it\'s an incident. Need to create ticket. Need severity: urgent. Could be "high". The user says "quiet urgent". So severity is high. We need to fill fields: assigned_to null, classification category INCIDENT_ANOMALY, confidence 1.0, severity "HIGH". sub_category maybe "Sales Report Discrepancy". extracted_entities maybe include "gross sales margin", "consolidated sales report", "SAP system". summary: "Gross sales margin discrepancy in consolidated sales report compared to SAP". raw_input Title? Not provided; Title maybe empty. Use Body as given. status maybe "OPEN". next_action maybe "Investigate discrepancy". response_message maybe null. ticket_id null. lastUpdateTime current timestamp? Could set to now. source maybe "User". Let\'s create ticket.', 'tool_calls': [{'id': 'fc_7b114aa5-e8f4-4413-96ce-6

In [15]:
print(response.additional_kwargs['reasoning_content'])

We need to decide if this is an incident. It's about gross sales margin discrepancy, urgent. It's not a follow_up or query, it's an incident. Need to create ticket. Need severity: urgent. Could be "high". The user says "quiet urgent". So severity is high. We need to fill fields: assigned_to null, classification category INCIDENT_ANOMALY, confidence 1.0, severity "HIGH". sub_category maybe "Sales Report Discrepancy". extracted_entities maybe include "gross sales margin", "consolidated sales report", "SAP system". summary: "Gross sales margin discrepancy in consolidated sales report compared to SAP". raw_input Title? Not provided; Title maybe empty. Use Body as given. status maybe "OPEN". next_action maybe "Investigate discrepancy". response_message maybe null. ticket_id null. lastUpdateTime current timestamp? Could set to now. source maybe "User". Let's create ticket.


In [8]:
## Node Functionality
def chatbot(state:State):
    return {"messages":[llm.invoke(state["messages"])]}

In [ ]:
graph_builder=StateGraph(State)

## Adding node
graph_builder.add_node("llmchatbot",chatbot)

## Adding Edges
graph_builder.add_edge(START,"llmchatbot")
graph_builder.add_edge("llmchatbot",END)

## compile the graph
graph=graph_builder.compile()

## Visualize the graph
from IPython.display import Image,display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    pass

In [19]:
response=graph.invoke({"messages":"what is machine learning/"})